# 🧠 Tumor Classif. with PyTorch⚡Lightning & MONAI ResNet 3D

The goal of this challenge is to Predict the status of a genetic biomarker important for brain cancer treatment.

All the code is refered from public repository: https://github.com/Borda/kaggle_brain-tumor-3D
Any nice contribution is welcome!

In [ ]:
! cp -r /kaggle/input/brain-tumor-classif-submissions/package_freeze/* package_freeze/

import glob, os, shutil
# finad all packages
pkgs = glob.glob("package_freeze/*.xyz")
# rename them back to correct name format
_= [shutil.move(p, p.replace(".xyz", ".tar.gz")) for p in pkgs]

! ls -l package_freeze/kaggle*
! pip install -q "kaggle_brain3D" --no-index --find-link package_freeze/

# ! pip install -q https://github.com/Borda/kaggle_brain-tumor-3D/archive/refs/heads/main.zip

! pip uninstall -q -y wandb
! pip list | grep torch

In [ ]:
! ls -l /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification
! nvidia-smi
! mkdir /kaggle/temp

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import kaggle_brain3d
print(kaggle_brain3d.__version__)

## Data exploration

These 3 cohorts are structured as follows: Each independent case has a dedicated folder identified by a five-digit number.
Within each of these “case” folders, there are four sub-folders, each of them corresponding to each of the structural multi-parametric MRI (mpMRI) scans, in DICOM format.
The exact mpMRI scans included are:

- **FLAIR**: Fluid Attenuated Inversion Recovery
- **T1w**: T1-weighted pre-contrast
- **T1Gd**: T1-weighted post-contrast
- **T2w**: T2-weighted

#### according to https://www.aapm.org/meetings/amos2/pdf/34-8205-79886-720.pdf

- T1: weighting weighting better deliniates deliniates anatomy anatomy
- T2: weighting weighting naturally naturally shows pathology

#### according to https://radiopaedia.org/articles/fluid-attenuated-inversion-recovery

Fluid attenuated inversion recovery (FLAIR) is a special inversion recovery sequence with a long inversion time. This removes signal from the cerebrospinal fluid in the resulting images 1. Brain tissue on FLAIR images appears similar to T2 weighted images with grey matter brighter than white matter but CSF is dark instead of bright.

To null the signal from fluid, the inversion time (TI) of the FLAIR pulse sequence is adjusted such that at equilibrium there is no net transverse magnetization of fluid.

The FLAIR sequence is part of almost all protocols for imaging the brain, particularly useful in the detection of subtle changes at the periphery of the hemispheres and in the periventricular region close to CSF.

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PATH_DATASET = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification"
PATH_MODELS = "../input/meidcalnet-pretrained-3d-resnet-weights"
PATH_TEMP = "/kaggle/temp"
SCAN_TYPES = ("FLAIR", "T1w", "T1CE", "T2w")

df_train = pd.read_csv(os.path.join(PATH_DATASET, "train_labels.csv"))
df_train["BraTS21ID"] = df_train["BraTS21ID"].apply(lambda i: "%05d" % i)
display(df_train.head())

See the dataset label distribution

In [ ]:
_= df_train["MGMT_value"].value_counts().plot(kind="pie", title="label distribution", autopct="%.1f%%")

For almost all scans we have all four types

In [ ]:
scans = [os.path.basename(p) for p in glob.glob(os.path.join(PATH_DATASET, "train", "*", "*"))]
_= pd.Series(scans).value_counts().plot(kind="bar", grid=True)

### Interactive view

showing particular scan in XYZ dimension/slices

In [ ]:
from ipywidgets import interact, IntSlider

from kaggle_brain3d.utils import load_volume, interpolate_volume, show_volume
from kaggle_brain3d.transforms import crop_volume


def interactive_show(volume_path: str, crop_thr: float):
    print(f"loading: {volume_path}")
    volume = load_volume(volume_path, percentile=0)
    print(f"sample shape: {volume.shape} >> {volume.dtype}")
    volume = interpolate_volume(volume)
    print(f"interp shape: {volume.shape} >> {volume.dtype}")
    volume = crop_volume(volume, crop_thr)
    print(f"crop shape: {volume.shape} >> {volume.dtype}")
    vol_shape = volume.shape
    interact(
        lambda x, y, z: plt.show(show_volume(volume, x, y, z)),
        x=IntSlider(min=0, max=vol_shape[0], step=5, value=int(vol_shape[0] / 2)),
        y=IntSlider(min=0, max=vol_shape[1], step=5, value=int(vol_shape[1] / 2)),
        z=IntSlider(min=0, max=vol_shape[2], step=5, value=int(vol_shape[2] / 2)),
    )


PATH_SAMPLE_VOLUME = os.path.join(PATH_DATASET, "train", "00005", "FLAIR")

interactive_show(PATH_SAMPLE_VOLUME, crop_thr=1e-6)

## Prepare dataset

### Pytorch Dataset

The basic building block is traforming raw data to Torch Dataset.
We have here loading particular DICOM images into a volume and saving as temp/cacher, so we do not need to take the very time demanding loading do next time - this boost the IO from about 2h to 8min

At the end we show a few sample images from prepared dataset.

In [ ]:
import os
import pandas as pd
import torch
from tqdm.auto import tqdm

from kaggle_brain3d.data import BrainScansDataset
from kaggle_brain3d.transforms import resize_volume

# ==============================

ds = BrainScansDataset(
    image_dir=os.path.join(PATH_DATASET, "train"),
    df_table=os.path.join(PATH_DATASET, "train_labels.csv"),
    crop_thr=None, cache_dir=PATH_TEMP,
)
for i in tqdm(range(2)):
    img = ds[i * 10]["data"]
    img = resize_volume(img[0])
    show_volume(img, fig_size=(9, 6))

### Lightning DataModule

It is constric to wrap all data-related peaces and define Pytoch dataloder for Training / Validation / Testing phase.

At the end we show a few sample images from the fost training batch.

In [ ]:
from functools import partial
import rising.transforms as rtr
from rising.loading import DataLoader, default_transform_call
from rising.random import DiscreteParameter, UniformParameter

from kaggle_brain3d.data import BrainScansDM, TRAIN_TRANSFORMS, VAL_TRANSFORMS
from kaggle_brain3d.transforms import RandomAffine, rising_zero_mean

# ==============================

dm = BrainScansDM(
    data_dir=PATH_DATASET,
    scan_types=["T2w"],
    vol_size=224,
    crop_thr=None,
    # crop_thr=1e-6,  # experimental crop threshold
    # batch_size=4,  # for full model training
    batch_size=6,  # for finetune head
    cache_dir=PATH_TEMP,
    # in_memory=True,
    num_workers=3,
    train_transforms=rtr.Compose(TRAIN_TRANSFORMS, transform_call=default_transform_call),
    valid_transforms=rtr.Compose(VAL_TRANSFORMS, transform_call=default_transform_call),
)
dm.prepare_data(num_proc=3)
dm.setup()
# dm.prepare_data(num_proc=3, dataset=dm.test_dataset)
print(f"Training batches: {len(dm.train_dataloader())}")
print(f"Validation batches: {len(dm.val_dataloader())}")
print(f"Test batches: {len(dm.test_dataloader())}")

# Quick view
for batch in dm.train_dataloader():
    for i in range(2):
        show_volume(batch["data"][i][0], fig_size=(6, 4), v_min_max=(-1., 3.))
    break

## Prepare 3D model

LightningModule is the core of PL, it wrappes all model related peaces, mainly:

- the model/architecture/weights
- evaluation metrics
- configs for optimizer and LR scheduler

In [ ]:
from torchsummary import summary
from kaggle_brain3d.models import LitBrainMRI, create_pretrained_medical_resnet
from monai.networks.nets import resnet10, resnet18, resnet34, resnet50, SEResNet50
from torch.optim import SGD, ASGD, SparseAdam

# ==============================


PATH_PRETRAINED_WEIGHTS = os.path.join(PATH_MODELS, "resnet_18_23dataset.pth")
net, pretraineds_layers = create_pretrained_medical_resnet(PATH_PRETRAINED_WEIGHTS, model_constructor=resnet18)

# net = SEResNet50(spatial_dims=3, in_channels=1, pretrained=True, num_classes=2)

model = LitBrainMRI(net=net, pretrained_params=pretraineds_layers, lr=5e-4)
# summary(model, input_size=(1, 128, 128, 128))

## Train a model

Lightning forces the following structure to your code which makes it reusable and shareable:

- Research code (the LightningModule).
- Engineering code (you delete, and is handled by the Trainer).
- Non-essential research code (logging, etc... this goes in Callbacks).
- Data (use PyTorch DataLoaders or organize them into a LightningDataModule).

Once you do this, you can train on multiple-GPUs, TPUs, CPUs and even in 16-bit precision without changing your code!

In [ ]:
import pytorch_lightning as pl
from kaggle_brain3d.models import FineTuneCB

logger = pl.loggers.CSVLogger(save_dir='logs/', name=model.name)
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=0.6)
fine = FineTuneCB(unfreeze_epoch=20)
ckpt = pl.callbacks.ModelCheckpoint(
    monitor='valid/auroc',
    save_top_k=1,
    save_last=True,
    filename='checkpoint/{epoch:02d}-{valid/auroc:.4f}',
    mode='max',
)

# ==============================

trainer = pl.Trainer(
    # overfit_batches=5,
    # fast_dev_run=True,
    gpus=1,
    callbacks=[ckpt],  #, fine, swa
    logger=logger,
    max_epochs=25,
    precision=16,
    benchmark=True,
    accumulate_grad_batches=8,
    # val_check_interval=0.5,
    progress_bar_refresh_rate=1,
    log_every_n_steps=5,
    weights_summary='top',
    auto_lr_find=True,
#     auto_scale_batch_size='binsearch',
)

# ==============================

# trainer.tune(
#     model, 
#     datamodule=dm, 
#     lr_find_kwargs=dict(min_lr=2e-5, max_lr=1e-3, num_training=10),
#     # scale_batch_size_kwargs=dict(max_trials=5),
# )
# print(f"Batch size: {dm.batch_size}")
# print(f"Learning Rate: {model.learning_rate}")

# ==============================

trainer.fit(model=model, datamodule=dm)

### Training progress

In [ ]:
metrics = pd.read_csv(f'{logger.log_dir}/metrics.csv')
display(metrics.head())

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)

df_metrics = pd.DataFrame(aggreg_metrics)
df_metrics[['train/loss', 'valid/loss']].plot(grid=True, legend=True, xlabel=agg_col)
df_metrics[['train/f1', 'train/auroc', 'valid/f1', 'valid/auroc']].plot(grid=True, legend=True, xlabel=agg_col)

## Predictions

In [ ]:
model.eval()
model.cpu()
submission = []

for batch in dm.test_dataloader():
    print(batch.keys())
    print(batch.get("label"))
    imgs = batch.get("data")
    print(imgs.shape)
    with torch.no_grad():
        preds = model(imgs)
    print(preds)
    probs = torch.nn.functional.softmax(preds)
    print(probs)
    break

In [ ]:
from kaggle_brain3d.models import make_submission

dm.batch_size = 2
df_submission = make_submission(model, dm.test_dataloader(), "cuda" if torch.cuda.is_available() else "cpu")
display(df_submission)
df_submission["MGMT_value"].to_csv("submission.csv")

In [ ]:
df_submission[["MGMT_value"]].hist(bins=25)

In [ ]:
! cat submission.csv